Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации.
2. Рассмотреть 2-а варианта:
* Инициализировать tf.keras.layers.Embedding предобученными векторами (зять к примеру с https://rusvectores.org/ru/.)
* Инициализировать слой tf.keras.layers.Embedding по умолчанию.
3. Сделать вывод

In [1]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install navec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

In [5]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from spacy.lang.ru.stop_words import STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from navec import Navec

In [6]:
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, get_file
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant

*****************************

In [8]:
MAX_WORDS = 2000
MAX_LEN = 100

In [9]:
data = pd.read_excel('/content/отзывы за лето.xls')
le = LabelEncoder()
data['Rating'] = le.fit_transform(data['Rating'])
data.sample(5)

,Rating,Content,Date
4580,4,"Очень удобно и быстро работать, спасибо большо...",2017-08-07
10859,4,Все понятно и легко.,2017-07-29
311,4,Отличное приложение сплошная экономия времени😎,2017-08-13
13944,4,норм,2017-07-24
6120,1,"Очень тупит, плохо заходит, вечная ошибка Плох...",2017-08-05


In [10]:
sw = set((get_stop_words("ru") + list(STOP_WORDS))) - {'не', 'ни', 'нет'}
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt, exclude=exclude, sw=sw, morpher=morpher):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
#     txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [11]:
data['preprocessed_text'] = data['Content'].apply(preprocess_text)

In [12]:
data.sample(5)

,Rating,Content,Date,preprocessed_text
9671,4,"Поставил из-за того, что SMS-ки стали приходит...",2017-07-31,поставить изз smsка приходить компьютер больши...
4078,4,Очень удобное приложение! И самое главное-неза...,2017-08-08,удобный приложение главноенезаменимоедажа пожа...
16166,3,"В целом неплохо; бывает, при входе с паролем д...",2017-07-19,целое неплохо вход пароль грузить выкидывать с...
3562,0,Такое же шовного как и весь охреневший сбербанк,2017-08-09,шовный охренесть сбербанк
16050,4,Все супер,2017-07-20,супер


In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
train_corpus = " ".join(data["preprocessed_text"])
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
len(tokens_filtered)

102261

In [16]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(MAX_WORDS-1)]
tokens_filtered_top[:7]

['приложение', 'не', 'удобно', 'работать', 'удобный', 'отлично', 'нравиться']

In [17]:
vocabulary = {v: k for k, v in enumerate(tokens_filtered_top, 1)}

In [18]:
len(vocabulary)

1999

In [19]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [20]:
data_train = np.asarray([text_to_sequence(text, MAX_LEN) for text in data['preprocessed_text']], dtype=np.int32)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data['Rating'], test_size=0.33)

*********************************

## Cлой Embedding по умолчанию

In [22]:
BATCH_SIZE = 1024
EPOCHES = 10

In [23]:
model = Sequential()
model.add(Embedding(input_dim=MAX_WORDS, output_dim=128, input_length=MAX_LEN))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [24]:
opt = Adam(learning_rate=0.001)
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer=opt,
              metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           82048     
                                                                 
 activation (Activation)     (None, 96, 128)           0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 20)                2580      
                                                                 
 activation_1 (Activation)   (None, 20)                0         
                                                        

In [26]:
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHES,
                    verbose=0,
                    validation_split=0.1)

In [27]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 2s 249ms/step - loss: 0.6509 - accuracy: 0.7775


Test loss: 0.6509063839912415
Test accuracy: 0.7775007486343384


***********************************************

## Embedding с предобученными векторами

In [30]:
path = '/content/navec_news_v1_1B_250K_300d_100q.tar'
emb = Navec.load(path)

In [31]:
embedding_dim = 300

embedding_matrix = np.zeros((MAX_WORDS, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = emb.get(word)
    if embedding_vector is not None:
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [32]:
embedding_matrix.shape

(2000, 300)

In [33]:
embedding_layer = Embedding(
    MAX_WORDS,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
)

In [34]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [35]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [36]:
history = model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHES,
          verbose=0,
          validation_split=0.1)

In [37]:
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 4s 559ms/step - loss: 0.7193 - accuracy: 0.7169


Test loss: 0.7193139791488647
Test accuracy: 0.7169257998466492


**Вывод:** использование предобученных векторов показало схожий результат.